<a href="https://colab.research.google.com/github/Marya1ZZ/UofT_ML_Datathon3/blob/notebooks/Datathon_3_Machine_learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch

# importing torch as t
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from torch.nn.functional import sigmoid, relu, tanh
from torch.optim import Adam
import matplotlib.pyplot as plt
import torch.nn.functional as F
import pandas as pd
from sklearn.preprocessing import StandardScaler
from torch.utils.data import TensorDataset, DataLoader

from torch.nn import Tanh, Linear, Sequential, Sigmoid, Dropout
#Use this if you are running the code on Colab
from google.colab import files
import io

uploaded = files.upload()

# Read the dataset into a Pandas DataFrame
data = pd.read_csv(next(iter(uploaded)))

# Display the first few rows of the dataset to ensure it's loaded correctly
data.head()
# checking the data types
data.dtypes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 20.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 64.7 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlink

Saving Datathon #3 Dataset - HAD7001.csv to Datathon #3 Dataset - HAD7001.csv


,0
encounter_id,int64
patient_id,int64
hospital_id,int64
hospital_death,int64
age,float64
...,...
leukemia,float64
lymphoma,float64
solid_tumor_with_metastasis,float64
apache_3j_bodysystem,object


In [ ]:
# Randomly sample 70% of the data for training
train_data = data.sample(frac = .7, random_state=10)

# Use the remaining 30% for testing
test_data = data.drop(train_data.index)

#data

# Data Preparation
from sklearn.preprocessing import LabelEncoder
from sklearn.impute import SimpleImputer

numerical_cols = ['age', 'bmi', 'd1_spo2_min', 'd1_sysbp_invasive_max', 'd1_sysbp_invasive_min',
                  'd1_heartrate_max', 'd1_temp_max', 'fio2_apache', 'd1_lactate_max', 'd1_creatinine_max',
                  'd1_glucose_max', 'd1_hco3_max', 'd1_inr_max']

categorical_cols = ['ethnicity', 'gender', 'readmission_status', 'immunosuppression', 'arf_apache', 'ventilated_apache', 'solid_tumor_with_metastasis',
                    'lymphoma', 'leukemia', 'hepatic_failure']

#data imputation
# Create an imputer for numerical columns (using median as an example)
num_imputer = SimpleImputer(strategy='median')  # You can change to 'mean' if needed

# Impute numerical columns in both train and test datasets
train_data[numerical_cols] = num_imputer.fit_transform(train_data[numerical_cols])
test_data[numerical_cols] = num_imputer.transform(test_data[numerical_cols])

# Create an imputer for categorical columns (using most frequent category)
cat_imputer = SimpleImputer(strategy='most_frequent')

# Impute categorical columns in both train and test datasets
train_data[categorical_cols] = cat_imputer.fit_transform(train_data[categorical_cols])
test_data[categorical_cols] = cat_imputer.transform(test_data[categorical_cols])

le = LabelEncoder()
train_data['ethnicity'] = le.fit_transform(train_data['ethnicity'])
test_data['ethnicity'] = le.transform(test_data['ethnicity'])

train_data['gender'] = le.fit_transform(train_data['gender'])
test_data['gender'] = le.transform(test_data['gender'])

train_data['icu_type'] = le.fit_transform(train_data['icu_type'])
test_data['icu_type'] = le.transform(test_data['icu_type'])

train_data['gcs'] = train_data['gcs_eyes_apache'] + train_data['gcs_motor_apache'] + train_data['gcs_verbal_apache']
test_data['gcs'] = test_data['gcs_eyes_apache'] + test_data['gcs_motor_apache'] + test_data['gcs_verbal_apache']

selected_columns = ['age', 'bmi', 'ethnicity', 'gender', 'readmission_status', 'gcs', 'icu_type',
                    'ventilated_apache', 'd1_spo2_min', 'd1_sysbp_invasive_max', 'd1_sysbp_invasive_min',
                    'd1_heartrate_max', 'd1_temp_max', 'arf_apache','fio2_apache', 'd1_lactate_max', 'd1_creatinine_max',
                    'd1_glucose_max', 'd1_hco3_max', 'd1_inr_max', 'immunosuppression', 'solid_tumor_with_metastasis', 'lymphoma', 'leukemia', 'hepatic_failure']



In [ ]:
 # Extract the 'hospital_death' column as the target variable for training and testing
y_train = train_data['hospital_death']
y_test = test_data['hospital_death']

# Extract the features (excluding 'hospital_death') for training and testing
X_train = train_data[selected_columns]
X_test = test_data[selected_columns]

# Standardize the features to have zero mean and unit variance
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

unique_values, counts = np.unique(y_train, return_counts=True)

print(unique_values)
print(counts)

[0 1]
[58667  5532]


In [ ]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((64199, 25), (64199,), (27514, 25), (27514,))

In [ ]:
unique_values, counts = np.unique(y_test, return_counts=True)
print(unique_values, counts)

[0 1] [25131  2383]


In [ ]:

X_train = np.nan_to_num(X_train)  # Replace NaNs with 0
X_test = np.nan_to_num(X_test)

y_train = np.nan_to_num(y_train)  # Replace NaNs with 0
y_test = np.nan_to_num(y_test)

batch_size = 8

# Convert them to tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)

X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

# Create data loaders
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset = TensorDataset(X_test_tensor, y_test_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Define the neural network model
class BinaryClassifier(nn.Module):
    def __init__(self, input_dim):
        super(BinaryClassifier, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 1)  # No Sigmoid here!
        )

    def forward(self, x):
        return self.model(x)  # No Sigmoid

# Initialize the model
input_dim = X_train.shape[1]
model = BinaryClassifier(input_dim)

criterion = nn.BCEWithLogitsLoss()  # This already applies Sigmoid internally
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 20
for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    correct, total = 0, 0

    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()

        outputs = model(X_batch)  # No Sigmoid needed

        # Compute the loss
        ###Debug
        #print(f"Outputs Min: {outputs.min().item()}, Max: {outputs.max().item()}")
        #print(f"Target Min: {y_batch.min().item()}, Max: {y_batch.max().item()}")

        loss = criterion(outputs, y_batch.float())  # Ensure y_batch is float32
        loss.backward()

        # Apply gradient clipping to prevent NaNs
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)

        optimizer.step()

        total_loss += loss.item()

        # Compute training accuracy
        predicted = (torch.sigmoid(outputs) > 0.5).float()  # Sigmoid for thresholding
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {total_loss/len(train_loader):.4f}, Accuracy: {correct/total:.4f}")

Epoch [1/20], Loss: 0.2265, Accuracy: 0.9226
Epoch [2/20], Loss: 0.2133, Accuracy: 0.9244
Epoch [3/20], Loss: 0.2121, Accuracy: 0.9243
Epoch [4/20], Loss: 0.2114, Accuracy: 0.9247
Epoch [5/20], Loss: 0.2108, Accuracy: 0.9250
Epoch [6/20], Loss: 0.2102, Accuracy: 0.9251
Epoch [7/20], Loss: 0.2096, Accuracy: 0.9244
Epoch [8/20], Loss: 0.2092, Accuracy: 0.9253
Epoch [9/20], Loss: 0.2091, Accuracy: 0.9253
Epoch [10/20], Loss: 0.2088, Accuracy: 0.9250
Epoch [11/20], Loss: 0.2084, Accuracy: 0.9254
Epoch [12/20], Loss: 0.2082, Accuracy: 0.9257
Epoch [13/20], Loss: 0.2080, Accuracy: 0.9256
Epoch [14/20], Loss: 0.2080, Accuracy: 0.9257
Epoch [15/20], Loss: 0.2081, Accuracy: 0.9256
Epoch [16/20], Loss: 0.2078, Accuracy: 0.9256
Epoch [17/20], Loss: 0.2077, Accuracy: 0.9261
Epoch [18/20], Loss: 0.2073, Accuracy: 0.9262
Epoch [19/20], Loss: 0.2075, Accuracy: 0.9257
Epoch [20/20], Loss: 0.2072, Accuracy: 0.9259


In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, confusion_matrix
from sklearn.metrics import classification_report

import torch.nn.functional as F

model.eval()

all_preds = []
all_probs = []
all_labels = []

total_loss = 0

with torch.no_grad():  # No gradient calculation needed for evaluation
    for X_batch, y_batch in test_loader:
        outputs = model(X_batch)  # Get raw logits

        # Apply sigmoid to get probabilities
        probs = torch.sigmoid(outputs).cpu().numpy()
        preds = (probs > 0.5).astype(int)  #binary predictions

        # Compute loss
        loss = criterion(outputs, y_batch.float())  #BCEWithLogitsLoss takes raw logits
        total_loss += loss.item()

        # Store results
        all_probs.extend(probs)
        all_preds.extend(preds)
        all_labels.extend(y_batch.cpu().numpy())

# Convert to numpy arrays
all_probs = np.array(all_probs).flatten()
all_preds = np.array(all_preds).flatten()
all_labels = np.array(all_labels).flatten()

# Compute evaluation metrics
test_loss = total_loss / len(test_loader)
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, zero_division=0)
recall = recall_score(all_labels, all_preds, zero_division=0)
f1 = f1_score(all_labels, all_preds, zero_division=0)
roc_auc = roc_auc_score(all_labels, all_probs)  # AUC probabilities
conf_matrix = confusion_matrix(all_labels, all_preds)

# Print evaluation results
print(f"Test Loss: {test_loss:.4f}")
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-score: {f1:.4f}")
print(f"ROC-AUC: {roc_auc:.4f}")
print("Confusion Matrix:")
print(conf_matrix)

# Generate classification report
report = classification_report(all_labels, all_preds, target_names=["No Death (0)", "Death (1)"])
print("Classification Report:")
print(report)



Test Loss: 0.2131
Accuracy: 0.9246
Precision: 0.7026
Recall: 0.2241
F1-score: 0.3398
ROC-AUC: 0.8600
Confusion Matrix:
[[24905   226]
 [ 1849   534]]
Classification Report:
              precision    recall  f1-score   support

No Death (0)       0.93      0.99      0.96     25131
   Death (1)       0.70      0.22      0.34      2383

    accuracy                           0.92     27514
   macro avg       0.82      0.61      0.65     27514
weighted avg       0.91      0.92      0.91     27514

